#### Feature Engineering  

Dataset: 
- _videogames_clean.csv_

Author: Luis Sergio Pastrana Lemus  
Date: 2025-05-29

# Feature engineering – Videogames Dataset

## __1. Libraries__

In [1]:
from pathlib import Path
import sys

# Define project root dynamically, gets the current directory from which the notebook belongs and moves one level upper
project_root = Path.cwd().parent

# Add src to sys.path if it is not already
if str(project_root) not in sys.path:

    sys.path.append(str(project_root))

# Import function directly (more controlled than import *)
from src import *

from functools import partial
from IPython.display import display, HTML
import numpy as np
import os
import pandas as pd

## __2. Path to Data file__

In [2]:
# Build route to data file and upload
data_file_path = project_root / "data" / "processed" / "clean"
df_vg = load_dataset_from_csv(data_file_path, "videogames_clean.csv", sep=',', header='infer')

In [3]:
# Format notebook output
format_notebook()

## 3 __Functions__

In [4]:
# Function for calculating ...

## 4 __Casting to data types__

In [5]:
# Call casting dtypes function from features.py
df_vg = cast_datatypes(df_vg)

In [6]:
# Adjust sales values for better visualization: original values (in millions) are scaled up by 1000
# to convert them to thousands, improving readability in graphs.
sales_columns = ['na_sales', 'eu_sales', 'jp_sales', 'other_sales']
df_vg[sales_columns] = df_vg[sales_columns] * 1000

## 4. Feature Engineering

### 4.1 Datasets

#### 4.1.1 DataSet videogames

In [7]:
df_vg

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,wii_sports,wii,2006.0,sports,41360.0,28960.0,3770.0,8450.0,76.0,8.00,E
1,super_mario_bros,nes,1985.0,platform,29080.0,3580.0,6810.0,770.0,NaN,NaN,NaN
2,mario_kart_wii,wii,2008.0,racing,15680.0,12760.0,3790.0,3290.0,82.0,8.30,E
3,wii_sports_resort,wii,2009.0,sports,15610.0,10930.0,3280.0,2950.0,80.0,8.00,E
4,pokemon_red_pokemon_blue,gb,1996.0,role_playing,11270.0,8890.0,10220.0,1000.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
16710,samurai_warriors_sanada_maru,ps3,2016.0,action,0.0,0.0,10.0,0.0,73.0,6.95,M
16711,lma_manager_2007,x360,2006.0,sports,0.0,10.0,0.0,0.0,72.0,7.30,E
16712,haitaka_no_psychedelica,psv,2016.0,adventure,0.0,0.0,10.0,0.0,72.0,7.80,M
16713,spirits_spells,gba,2003.0,platform,10.0,0.0,0.0,0.0,69.0,7.85,E


In [8]:
df_vg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   name             16715 non-null  string  
 1   platform         16715 non-null  category
 2   year_of_release  16713 non-null  float64 
 3   genre            16715 non-null  category
 4   na_sales         16715 non-null  float64 
 5   eu_sales         16715 non-null  float64 
 6   jp_sales         16715 non-null  float64 
 7   other_sales      16715 non-null  float64 
 8   critic_score     15360 non-null  float32 
 9   user_score       15185 non-null  float32 
 10  rating           15390 non-null  object  
dtypes: category(2), float32(2), float64(5), object(1), string(1)
memory usage: 1.1+ MB


In [9]:
# Sales variation per platform
df_vg_platform_sales = df_vg.loc[:, ['platform', 'na_sales', 'eu_sales', 'jp_sales', 'other_sales']]
df_vg_platform_sales = df_vg.groupby(['platform'], observed=False).agg({'na_sales':'sum', 'eu_sales':'sum', 'jp_sales':'sum', 
                                                                        'other_sales':'sum'}).reset_index()
df_vg_platform_sales

,platform,na_sales,eu_sales,jp_sales,other_sales
0,2600,90600.0,5470.0,0.0,910.0
1,3do,0.0,0.0,100.0,0.0
2,3ds,83490.0,61480.0,100670.0,13360.0
3,dc,5430.0,1690.0,8560.0,270.0
4,ds,382400.0,188890.0,175570.0,59260.0
...,...,...,...,...,...
26,wiiu,38100.0,25130.0,13010.0,5950.0
27,ws,0.0,0.0,1420.0,0.0
28,x360,602470.0,270760.0,12430.0,85760.0
29,xb,186690.0,60950.0,1380.0,8720.0


In [10]:
df_vg_platform_sales['total_sales'] = df_vg_platform_sales[['na_sales', 'eu_sales', 'jp_sales', 'other_sales']].sum(axis=1)
df_vg_platform_sales

,platform,na_sales,eu_sales,jp_sales,other_sales,total_sales
0,2600,90600.0,5470.0,0.0,910.0,96980.0
1,3do,0.0,0.0,100.0,0.0,100.0
2,3ds,83490.0,61480.0,100670.0,13360.0,259000.0
3,dc,5430.0,1690.0,8560.0,270.0,15950.0
4,ds,382400.0,188890.0,175570.0,59260.0,806120.0
...,...,...,...,...,...,...
26,wiiu,38100.0,25130.0,13010.0,5950.0,82190.0
27,ws,0.0,0.0,1420.0,0.0,1420.0
28,x360,602470.0,270760.0,12430.0,85760.0,971420.0
29,xb,186690.0,60950.0,1380.0,8720.0,257740.0


In [11]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "feature" / "vg_platform_sales_feature.csv"

df_vg_platform_sales.to_csv(processed_path, index=False)

In [12]:
# Platforms highest total sales per year
df_vg_platform_top_sales_year = df_vg.loc[:, ['platform', 'year_of_release', 'na_sales', 'eu_sales', 'jp_sales', 'other_sales']]
df_vg_platform_top_sales_year = df_vg.groupby(['platform', 'year_of_release'], observed=False).agg({'na_sales':'sum', 'eu_sales':'sum', 
                                                                                                    'jp_sales':'sum', 'other_sales':'sum'}).reset_index()

cols = ['na_sales', 'eu_sales', 'jp_sales', 'other_sales']

df_vg_platform_top_sales_year = df_vg_platform_top_sales_year.loc[~(df_vg_platform_top_sales_year[cols] == 0).any(axis=1)]
df_vg_platform_top_sales_year

,platform,year_of_release,na_sales,eu_sales,jp_sales,other_sales
124,3ds,2011.0,27640.0,18840.0,12830.0,4450.0
125,3ds,2012.0,17110.0,11680.0,20030.0,2590.0
126,3ds,2013.0,15560.0,14840.0,23570.0,2600.0
127,3ds,2014.0,12330.0,8730.0,20690.0,2010.0
128,3ds,2015.0,6170.0,5060.0,15520.0,1030.0
...,...,...,...,...,...,...
1333,xb,2005.0,35110.0,12560.0,20.0,1740.0
1386,xone,2013.0,11850.0,5340.0,20.0,1750.0
1387,xone,2014.0,30970.0,18150.0,140.0,4810.0
1388,xone,2015.0,36030.0,18470.0,170.0,5470.0


In [13]:
df_vg_platform_top_sales_year['total_sales'] = df_vg_platform_top_sales_year[['na_sales', 'eu_sales', 'jp_sales', 'other_sales']].sum(axis=1)
df_vg_platform_top_sales_year

,platform,year_of_release,na_sales,eu_sales,jp_sales,other_sales,total_sales
124,3ds,2011.0,27640.0,18840.0,12830.0,4450.0,63760.0
125,3ds,2012.0,17110.0,11680.0,20030.0,2590.0,51410.0
126,3ds,2013.0,15560.0,14840.0,23570.0,2600.0,56570.0
127,3ds,2014.0,12330.0,8730.0,20690.0,2010.0,43760.0
128,3ds,2015.0,6170.0,5060.0,15520.0,1030.0,27780.0
...,...,...,...,...,...,...,...
1333,xb,2005.0,35110.0,12560.0,20.0,1740.0,49430.0
1386,xone,2013.0,11850.0,5340.0,20.0,1750.0,18960.0
1387,xone,2014.0,30970.0,18150.0,140.0,4810.0,54070.0
1388,xone,2015.0,36030.0,18470.0,170.0,5470.0,60140.0


In [14]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "feature" / "vg_platform_top_sales_year_feature.csv"

df_vg_platform_top_sales_year.to_csv(processed_path, index=False)

In [15]:
df_vg.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales', 'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'], dtype='object')

In [16]:
# Platforms that used to be popular but are now losing sales
df_popular_loss = df_vg.loc[: , ['name', 'platform', 'year_of_release', 'na_sales', 'eu_sales', 'jp_sales', 'other_sales']]
df_popular_loss

,name,platform,year_of_release,na_sales,eu_sales,jp_sales,other_sales
0,wii_sports,wii,2006.0,41360.0,28960.0,3770.0,8450.0
1,super_mario_bros,nes,1985.0,29080.0,3580.0,6810.0,770.0
2,mario_kart_wii,wii,2008.0,15680.0,12760.0,3790.0,3290.0
3,wii_sports_resort,wii,2009.0,15610.0,10930.0,3280.0,2950.0
4,pokemon_red_pokemon_blue,gb,1996.0,11270.0,8890.0,10220.0,1000.0
...,...,...,...,...,...,...,...
16710,samurai_warriors_sanada_maru,ps3,2016.0,0.0,0.0,10.0,0.0
16711,lma_manager_2007,x360,2006.0,0.0,10.0,0.0,0.0
16712,haitaka_no_psychedelica,psv,2016.0,0.0,0.0,10.0,0.0
16713,spirits_spells,gba,2003.0,10.0,0.0,0.0,0.0


In [17]:
columns = ['na_sales', 'eu_sales', 'jp_sales', 'other_sales']
df_popular_loss['total_sales'] = df_popular_loss[columns].sum(axis=1)
df_popular_loss

,name,platform,year_of_release,na_sales,eu_sales,jp_sales,other_sales,total_sales
0,wii_sports,wii,2006.0,41360.0,28960.0,3770.0,8450.0,82540.0
1,super_mario_bros,nes,1985.0,29080.0,3580.0,6810.0,770.0,40240.0
2,mario_kart_wii,wii,2008.0,15680.0,12760.0,3790.0,3290.0,35520.0
3,wii_sports_resort,wii,2009.0,15610.0,10930.0,3280.0,2950.0,32770.0
4,pokemon_red_pokemon_blue,gb,1996.0,11270.0,8890.0,10220.0,1000.0,31380.0
...,...,...,...,...,...,...,...,...
16710,samurai_warriors_sanada_maru,ps3,2016.0,0.0,0.0,10.0,0.0,10.0
16711,lma_manager_2007,x360,2006.0,0.0,10.0,0.0,0.0,10.0
16712,haitaka_no_psychedelica,psv,2016.0,0.0,0.0,10.0,0.0,10.0
16713,spirits_spells,gba,2003.0,10.0,0.0,0.0,0.0,10.0


In [18]:
df_popular_loss = df_popular_loss.drop(columns=['na_sales', 'eu_sales', 'jp_sales', 'other_sales'])
df_popular_loss

,name,platform,year_of_release,total_sales
0,wii_sports,wii,2006.0,82540.0
1,super_mario_bros,nes,1985.0,40240.0
2,mario_kart_wii,wii,2008.0,35520.0
3,wii_sports_resort,wii,2009.0,32770.0
4,pokemon_red_pokemon_blue,gb,1996.0,31380.0
...,...,...,...,...
16710,samurai_warriors_sanada_maru,ps3,2016.0,10.0
16711,lma_manager_2007,x360,2006.0,10.0
16712,haitaka_no_psychedelica,psv,2016.0,10.0
16713,spirits_spells,gba,2003.0,10.0


In [19]:
df_popular_loss = df_popular_loss.groupby(['platform', 'year_of_release'], observed=False).agg({'name': 'count', 'total_sales': 'sum'})
df_popular_loss.columns = ['games', 'total_sales']
df_popular_loss

games  total_sales
platform year_of_release                    
2600     1977.0               5       2860.0
         1978.0               2        640.0
         1979.0               1        310.0
         1980.0              14      16490.0
         1981.0              47      36120.0
...                         ...          ...
xone     2017.0               0          0.0
         2019.0               0          0.0
         2021.0               0          0.0
         2022.0               0          0.0
         2023.0               0          0.0

[1395 rows x 2 columns]

In [20]:
df_popular_loss = df_popular_loss.loc[~(df_popular_loss['games'] == 0) & ~(df_popular_loss['total_sales'] == 0)].reset_index()
df_popular_loss

,platform,year_of_release,games,total_sales
0,2600,1977.0,5,2860.0
1,2600,1978.0,2,640.0
2,2600,1979.0,1,310.0
3,2600,1980.0,14,16490.0
4,2600,1981.0,47,36120.0
...,...,...,...,...
270,xb,2022.0,1,310.0
271,xone,2013.0,19,18960.0
272,xone,2014.0,61,54070.0
273,xone,2015.0,80,60140.0


In [21]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "feature" / "vg_platform_popular_loss_feature.csv"

df_popular_loss.to_csv(processed_path, index=False)

In [22]:
# Platform profitability
mask = (df_vg['year_of_release'] > 1993) & (df_vg['year_of_release'] < 2018)
df_platform_profit = df_vg.loc[mask, ['name', 'platform', 'year_of_release', 'na_sales', 'eu_sales', 'jp_sales', 'other_sales']]
df_platform_profit['total_sales'] = df_platform_profit[['na_sales', 'eu_sales', 'jp_sales', 'other_sales']].sum(axis=1)
df_platform_profit


,name,platform,year_of_release,na_sales,eu_sales,jp_sales,other_sales,total_sales
0,wii_sports,wii,2006.0,41360.0,28960.0,3770.0,8450.0,82540.0
2,mario_kart_wii,wii,2008.0,15680.0,12760.0,3790.0,3290.0,35520.0
3,wii_sports_resort,wii,2009.0,15610.0,10930.0,3280.0,2950.0,32770.0
4,pokemon_red_pokemon_blue,gb,1996.0,11270.0,8890.0,10220.0,1000.0,31380.0
6,new_super_mario_bros,ds,2006.0,11280.0,9140.0,6500.0,2880.0,29800.0
...,...,...,...,...,...,...,...,...
16710,samurai_warriors_sanada_maru,ps3,2016.0,0.0,0.0,10.0,0.0,10.0
16711,lma_manager_2007,x360,2006.0,0.0,10.0,0.0,0.0,10.0
16712,haitaka_no_psychedelica,psv,2016.0,0.0,0.0,10.0,0.0,10.0
16713,spirits_spells,gba,2003.0,10.0,0.0,0.0,0.0,10.0


In [23]:
df_lifespan = df_platform_profit.groupby('platform', observed=False)['year_of_release'].agg(['min', 'max']).reset_index()
df_lifespan['lifespan'] = df_lifespan['max'] - df_lifespan['min'] + 1
df_lifespan

,platform,min,max,lifespan
0,2600,NaN,NaN,NaN
1,3do,1994.0,1995.0,2.0
2,3ds,2010.0,2016.0,7.0
3,dc,1998.0,2008.0,11.0
4,ds,2001.0,2014.0,14.0
...,...,...,...,...
26,wiiu,2012.0,2016.0,5.0
27,ws,1999.0,2001.0,3.0
28,x360,2001.0,2016.0,16.0
29,xb,2000.0,2008.0,9.0


In [24]:
# Relative Profitability (total_sales/games_sold/life_span)
df_platform_profit = df_platform_profit.groupby('platform', observed=True).agg({'total_sales': 'sum', 'name': 'count'}).reset_index()
df_platform_profit = df_platform_profit.rename(columns={'name': 'games_sold'})
df_platform_profit

,platform,total_sales,games_sold
0,3do,100.0,3
1,3ds,259000.0,520
2,dc,15950.0,52
3,ds,805880.0,2147
4,gb,153110.0,68
...,...,...,...
24,wiiu,82190.0,147
25,ws,1420.0,6
26,x360,970970.0,1259
27,xb,257370.0,822


In [25]:
df_platform_profit = df_platform_profit.merge(df_lifespan, on='platform')
df_platform_profit = df_platform_profit.drop(columns=['min', 'max'])
df_platform_profit

,platform,total_sales,games_sold,lifespan
0,3do,100.0,3,2.0
1,3ds,259000.0,520,7.0
2,dc,15950.0,52,11.0
3,ds,805880.0,2147,14.0
4,gb,153110.0,68,8.0
...,...,...,...,...
24,wiiu,82190.0,147,5.0
25,ws,1420.0,6,3.0
26,x360,970970.0,1259,16.0
27,xb,257370.0,822,9.0


In [26]:
df_platform_profit['profit'] = round(df_platform_profit['total_sales'] / df_platform_profit['games_sold']/ df_platform_profit['lifespan'], 2)
df_platform_profit

,platform,total_sales,games_sold,lifespan,profit
0,3do,100.0,3,2.0,16.67
1,3ds,259000.0,520,7.0,71.15
2,dc,15950.0,52,11.0,27.88
3,ds,805880.0,2147,14.0,26.81
4,gb,153110.0,68,8.0,281.45
...,...,...,...,...,...
24,wiiu,82190.0,147,5.0,111.82
25,ws,1420.0,6,3.0,78.89
26,x360,970970.0,1259,16.0,48.20
27,xb,257370.0,822,9.0,34.79


In [27]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "feature" / "vg_platform_profit_feature.csv"

df_platform_profit.to_csv(processed_path, index=False)

In [28]:
# Average sales across various platforms
mask = (df_vg['year_of_release'] > 1993) & (df_vg['year_of_release'] < 2018)
df_vg_platform_avg_sales = df_vg.loc[mask, ['platform', 'na_sales', 'eu_sales', 'jp_sales', 'other_sales']]
df_vg_platform_avg_sales['total_sales'] = df_vg_platform_avg_sales[['na_sales', 'eu_sales', 'jp_sales', 'other_sales']].sum(axis=1)
df_vg_platform_avg_sales


,platform,na_sales,eu_sales,jp_sales,other_sales,total_sales
0,wii,41360.0,28960.0,3770.0,8450.0,82540.0
2,wii,15680.0,12760.0,3790.0,3290.0,35520.0
3,wii,15610.0,10930.0,3280.0,2950.0,32770.0
4,gb,11270.0,8890.0,10220.0,1000.0,31380.0
6,ds,11280.0,9140.0,6500.0,2880.0,29800.0
...,...,...,...,...,...,...
16710,ps3,0.0,0.0,10.0,0.0,10.0
16711,x360,0.0,10.0,0.0,0.0,10.0
16712,psv,0.0,0.0,10.0,0.0,10.0
16713,gba,10.0,0.0,0.0,0.0,10.0


In [29]:
df_vg_platform_avg_sales = df_vg_platform_avg_sales.groupby('platform', observed=False).agg({'na_sales': 'median', 'eu_sales': 'median', 
                                                                                             'jp_sales': 'median', 'other_sales': 'median', 
                                                                                             'total_sales': 'median'})
df_vg_platform_avg_sales = df_vg_platform_avg_sales.dropna().reset_index()
df_vg_platform_avg_sales

,platform,na_sales,eu_sales,jp_sales,other_sales,total_sales
0,3do,0.0,0.0,20.0,0.0,20.0
1,3ds,10.0,0.0,50.0,0.0,120.0
2,dc,0.0,0.0,120.0,0.0,135.0
3,ds,60.0,0.0,0.0,10.0,110.0
4,gb,0.0,0.0,375.0,0.0,945.0
...,...,...,...,...,...,...
24,wiiu,110.0,70.0,0.0,20.0,220.0
25,ws,0.0,0.0,215.0,0.0,215.0
26,x360,170.0,50.0,0.0,20.0,280.0
27,xb,110.0,30.0,0.0,10.0,150.0


In [30]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "feature" / "vg_platform_avg_sales_feature.csv"

df_vg_platform_avg_sales.to_csv(processed_path, index=False)

In [31]:
df_vg.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales', 'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'], dtype='object')

In [32]:
# Impact of User and Critic Reviews on Game Sales for [Platform]
mask = (df_vg['year_of_release'] > 1993) & (df_vg['year_of_release'] < 2018)
df_vg_us_cs_sales = df_vg.loc[mask, ['platform', 'na_sales', 'eu_sales', 'jp_sales', 'other_sales', 'user_score', 'critic_score']]
df_vg_us_cs_sales['total_sales'] = df_vg_us_cs_sales[['na_sales', 'eu_sales', 'jp_sales', 'other_sales']].sum(axis=1)
df_vg_us_cs_sales

,platform,na_sales,eu_sales,jp_sales,other_sales,user_score,critic_score,total_sales
0,wii,41360.0,28960.0,3770.0,8450.0,8.00,76.0,82540.0
2,wii,15680.0,12760.0,3790.0,3290.0,8.30,82.0,35520.0
3,wii,15610.0,10930.0,3280.0,2950.0,8.00,80.0,32770.0
4,gb,11270.0,8890.0,10220.0,1000.0,NaN,NaN,31380.0
6,ds,11280.0,9140.0,6500.0,2880.0,8.50,89.0,29800.0
...,...,...,...,...,...,...,...,...
16710,ps3,0.0,0.0,10.0,0.0,6.95,73.0,10.0
16711,x360,0.0,10.0,0.0,0.0,7.30,72.0,10.0
16712,psv,0.0,0.0,10.0,0.0,7.80,72.0,10.0
16713,gba,10.0,0.0,0.0,0.0,7.85,69.0,10.0


In [33]:
df_vg_us_sales = df_vg_us_cs_sales.groupby(['platform', 'user_score'], observed=False).agg({'na_sales': 'sum', 'eu_sales': 'sum', 'jp_sales': 'sum', 
                                                                                            'other_sales': 'sum', 'total_sales': 'sum'})
cols = ['na_sales', 'eu_sales', 'jp_sales', 'other_sales', 'total_sales']
df_vg_us_sales = df_vg_us_sales.loc[~(df_vg_us_sales[cols] == 0).all(axis=1), :].reset_index()
df_vg_us_sales

,platform,user_score,na_sales,eu_sales,jp_sales,other_sales,total_sales
0,3ds,2.4,100.0,350.0,0.0,30.0,480.0
1,3ds,2.8,0.0,30.0,0.0,0.0,30.0
2,3ds,3.0,150.0,150.0,0.0,30.0,330.0
3,3ds,3.3,300.0,370.0,30.0,70.0,770.0
4,3ds,3.4,160.0,100.0,0.0,20.0,280.0
...,...,...,...,...,...,...,...
1506,xone,8.6,260.0,90.0,0.0,40.0,390.0
1507,xone,8.7,130.0,60.0,0.0,20.0,210.0
1508,xone,8.8,10.0,0.0,0.0,0.0,10.0
1509,xone,8.9,30.0,10.0,0.0,0.0,40.0


In [34]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "feature" / "vg_us_sales_feature.csv"

df_vg_us_sales.to_csv(processed_path, index=False)

In [35]:
df_vg_cs_sales = df_vg_us_cs_sales.groupby(['platform', 'critic_score'], observed=False).agg({'na_sales': 'sum', 'eu_sales': 'sum', 'jp_sales': 'sum', 
                                                                                            'other_sales': 'sum', 'total_sales': 'sum'})
cols = ['na_sales', 'eu_sales', 'jp_sales', 'other_sales', 'total_sales']
df_vg_cs_sales = df_vg_cs_sales.loc[~(df_vg_cs_sales[cols] == 0).all(axis=1), :].reset_index()
df_vg_cs_sales

,platform,critic_score,na_sales,eu_sales,jp_sales,other_sales,total_sales
0,3ds,26.0,10.0,10.0,0.0,0.0,20.0
1,3ds,34.0,0.0,30.0,0.0,0.0,30.0
2,3ds,38.0,0.0,10.0,0.0,0.0,10.0
3,3ds,42.0,90.0,30.0,0.0,10.0,130.0
4,3ds,43.0,370.0,250.0,20.0,50.0,690.0
...,...,...,...,...,...,...,...
1572,xone,90.0,860.0,60.0,0.0,110.0,1030.0
1573,xone,91.0,2010.0,1660.0,0.0,340.0,4010.0
1574,xone,94.0,410.0,270.0,0.0,70.0,750.0
1575,xone,95.0,410.0,260.0,10.0,70.0,750.0


In [36]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "feature" / "vg_cs_sales_feature.csv"

df_vg_cs_sales.to_csv(processed_path, index=False)

In [64]:
# Sales Comparison of the Same Games Across Different Platforms
mask = (df_vg['year_of_release'] > 1993) & (df_vg['year_of_release'] < 2018)
df_vg_games_platform = df_vg.loc[mask, :]
df_vg_games_platform = df_vg_games_platform.groupby(['name', 'platform'], observed=False).agg({'na_sales': 'sum', 'eu_sales': 'sum', 
                                                                                               'jp_sales': 'sum', 'other_sales': 'sum'})

df_vg_games_platform = df_vg_games_platform.loc[~(df_vg_games_platform['na_sales'] == 0.0) & 
                                                ~(df_vg_games_platform['eu_sales'] == 0.0) &
                                                ~(df_vg_games_platform['jp_sales'] == 0.0) &
                                                ~(df_vg_games_platform['other_sales'] == 0.0), :]

df_name_counts = df_vg_games_platform.groupby(level='name').size().rename('name_total_count')
df_vg_games_platform = df_vg_games_platform.join(df_name_counts, on='name')

df_vg_games_platform

na_sales  eu_sales  jp_sales  other_sales  name_total_count
name                              platform                                                             
007_quantum_of_solace             ps3          430.0     510.0      20.0        190.0                 3
                                  wii          290.0     280.0      10.0         70.0                 3
                                  x360         820.0     510.0      10.0        140.0                 3
007_the_world_is_not_enough       n64         1130.0     380.0      20.0         30.0                 1
1080_teneighty_snowboarding       n64         1250.0     610.0     130.0         50.0                 1
...                                              ...       ...       ...          ...               ...
zone_of_the_enders_hd_collection  x360         140.0      30.0      10.0         20.0                 2
zone_of_the_enders_the_2nd_runner ps2          150.0     120.0     110.0         40.0                 1
zoo_keeper                        ds           100.0      10.0      50.0         10.0                 1
zoo_resort_3d                     3ds          110.0      90.0      30.0         20.0                 1
zoo_tycoon_ds                     ds           860.0      30.0      10.0         80.0                 1

[2287 rows x 5 columns]

In [65]:
df_vg_games_platform = df_vg_games_platform.loc[(df_vg_games_platform['name_total_count'] > 1), :].reset_index()
df_vg_games_platform

,name,platform,na_sales,eu_sales,jp_sales,other_sales,name_total_count
0,007_quantum_of_solace,ps3,430.0,510.0,20.0,190.0,3
1,007_quantum_of_solace,wii,290.0,280.0,10.0,70.0,3
2,007_quantum_of_solace,x360,820.0,510.0,10.0,140.0,3
3,2010_fifa_world_cup_south_africa,ps3,300.0,640.0,70.0,220.0,4
4,2010_fifa_world_cup_south_africa,psp,90.0,220.0,30.0,120.0,4
...,...,...,...,...,...,...,...
769,x_blades,x360,100.0,20.0,20.0,10.0,2
770,xenoblade_chronicles,3ds,270.0,140.0,120.0,40.0,2
771,xenoblade_chronicles,wii,430.0,250.0,160.0,70.0,2
772,zone_of_the_enders_hd_collection,ps3,210.0,70.0,90.0,50.0,2


In [66]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "feature" / "vg_games_platform_feature.csv"

df_vg_games_platform.to_csv(processed_path, index=False)

In [72]:
# Genre Distribution Across the Video Game Market.
mask = (df_vg['year_of_release'] > 1993) & (df_vg['year_of_release'] < 2018)
df_vg_games_genre = df_vg.loc[mask, :]
df_vg_games_genre = df_vg_games_genre.groupby(['genre'], observed=False).agg({'na_sales': 'sum', 'eu_sales': 'sum', 
                                                                              'jp_sales': 'sum', 'other_sales': 'sum'})
df_vg_games_genre = df_vg_games_genre.loc[~(df_vg_games_genre['na_sales'] == 0.0) & 
                                          ~(df_vg_games_genre['eu_sales'] == 0.0) &
                                          ~(df_vg_games_genre['jp_sales'] == 0.0) &
                                          ~(df_vg_games_genre['other_sales'] == 0.0), :]
df_vg_games_genre['total_sales'] = df_vg_games_genre[['na_sales', 'eu_sales', 'jp_sales', 'other_sales']].sum(axis=1)
df_vg_games_genre = df_vg_games_genre.reset_index()
df_vg_games_genre

,genre,na_sales,eu_sales,jp_sales,other_sales,total_sales
0,action,827500.0,511360.0,148450.0,183520.0,1670830.0
1,adventure,94340.0,59820.0,46610.0,16060.0,216830.0
2,fighting,210420.0,96700.0,72310.0,35810.0,415240.0
3,misc,401310.0,211710.0,100810.0,74220.0,788050.0
4,platform,334770.0,176720.0,84190.0,47130.0,642810.0
5,puzzle,72050.0,42910.0,35730.0,10870.0,161560.0
6,racing,344750.0,232310.0,46940.0,75180.0,699180.0
7,role_playing,329180.0,188550.0,323330.0,57930.0,898990.0
8,shooter,534920.0,313330.0,31810.0,103120.0,983180.0
9,simulation,178770.0,112150.0,62220.0,30540.0,383680.0


In [73]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "feature" / "vg_games_genre_feature.csv"

df_vg_games_genre.to_csv(processed_path, index=False)